In [ ]:
import os
import torch
from datasets import load_dataset, Dataset
import pandas as pd
import transformers
from transformers import AutoTokenizer
from trl import SFTTrainer
import transformers
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from pynvml import *
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import glob

In [ ]:
directory_path = r'/home/sdm/Desktop/shakib/KUET LLM/data/'
file_pattern = '*.docx'  
file_paths = glob.glob(directory_path + file_pattern)
file_paths

In [ ]:
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

In [ ]:
all_doc=[]
for x in file_paths:
    loader = Docx2txtLoader(x)
    documents=loader.load()
    # print(((documents)[0]).page_content)
    all_doc.append(str(documents[0].page_content))
docs='\n\n'.join(all_doc)

In [ ]:
RAG.index(
    collection=[docs],
    index_name="mistral_colbert",
    max_document_length=256,
    split_documents=True,
)

In [ ]:
retriever = RAG.as_langchain_retriever(k=3)

In [ ]:
from langchain import HuggingFacePipeline
# path='google/gemma-7b'
# path="KUETLLM_zepyr"
# path='mistralai/Mistral-7B-Instruct-v0.1'
# path="HuggingFaceH4/zephyr-7b-beta"
# path="NousResearch/Llama-2-7b-chat-hf"
# path="models/full_KUET_LLM_zepyhr"
path="models/full_KUET_LLM_Mistral"
model_ans="model_ans_mistral_finetuned_486_colbert"
tokenizer = AutoTokenizer.from_pretrained(path,
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained(path,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

# model = AutoModelForCausalLM.from_pretrained(
#     path,
#     # quantization_config=bnb_config,
#     device_map="auto",
#     trust_remote_code=True,
#     attn_implementation="flash_attention_2",
#     torch_dtype=torch.bfloat16,

# )
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )
    
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})


In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# torch.cuda.empty_cache()

In [ ]:
def ans_ret(inp,history):
    ans=rag_chain.invoke(inp)
    k=ans.split("Based on the text material")
    k2=ans.split("Hope that helped! Let me know if you have any more questions.")

    if len(k)>=2:
        k3=k[0].split("Hope that helped! Let me know if you have any more questions.")
        if len(k3)>=2:
            return k3[0]
        else:
            return k[0]
    if len(k2)>=2:
        return k2[0]
    return ans

In [ ]:
import pandas as pd
import tqdm
dat=[]
cnt=0
df=pd.read_excel(r"data/KUET information2k20.xlsx")
df.head()
for id,ques in tqdm.tqdm(zip(df['id'],df['Question'])):
    ans=ans_ret(ques,"")
    dat.append({
        "id":id,
        "question":ques,
        "answer":ans
    })
    # if cnt==5:
    #     break
    # cnt+=1
    

In [ ]:
sa=pd.DataFrame(dat)
sa.to_excel(f"data//{model_ans}.xlsx",index=False)